In [1]:
from random import choice, randint, shuffle
from keras.models import Sequential
from keras.layers import Dense

from itertools import chain

import numpy as np
from string import ascii_lowercase

alph = ascii_lowercase + ' '

maxsize = 18

Using TensorFlow backend.


In [9]:
words = set(list(w.strip() for w in open('real.txt')))

def isreal(word):
    return word in words

twords = tuple(words)


def realword() -> str:
    return choice(twords)

ff = open('fake.txt', 'w')

for _ in range(len(twords)):
    ff.write(''.join(choice(ascii_lowercase) for _ in range(len(realword())))+'\n')

In [55]:
# from time import time
# 
# r = tuple(open('RandomWords/real.txt'))
# f = tuple(open('RandomWords/fake.txt'))
# 
# def splatify(word):
#     return tuple(chain(*[[int(c==alph[i]) for i in range(len(alph))] for c in word]))
# 
# start = time()
# w = np.array([
#                  splatify(
#                      list((' '+word.strip()).rjust(maxsize))
#                  ) 
#                  for word in r+f])
# 
# 
# a = [1]*len(r) + [0]*len(f)
# print(time()-start)

2.8811020851135254


In [10]:
from time import time

r = tuple(open('real.txt'))
f = tuple(open('fake.txt'))

# @numba.jit
def splatify(word):
    word = list(' '+word.strip().rjust(maxsize))
    t = np.zeros(27*len(word))
    
    for i,char in enumerate(word):
        if char not in alph:
            print(word)
        t[i*27 + alph.index(char)] = 1
    
    return t
    
#     
# for word in f:
#     print(word)
#     for char in word.strip():
#         if char not in alph:
#             print(char)

start = time()
w = np.array([
                 splatify(word)
                 for word in r+f])


a = [1]*len(r) + [0]*len(f)
print(time()-start)

3.516174077987671


In [11]:
model = Sequential([
    Dense(250, activation='relu', input_dim=len(splatify('test'))),
    Dense(50, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


assert len(w) == len(a)

model.fit(w, a, epochs=3, batch_size=100, verbose=2)

# scores = model.evaluate(w, a)
# print(f"\nacc: {scores[1]*100:.2f}%")

Epoch 1/3
 - 6s - loss: 0.1421 - acc: 0.9458
Epoch 2/3
 - 5s - loss: 0.0683 - acc: 0.9747
Epoch 3/3
 - 5s - loss: 0.0473 - acc: 0.9826


In [12]:
def predict(x:str) -> float:
    t = np.array(splatify(x))
    t = t.reshape((1 , 513))
    return model.predict(t)[0][0]

real_set = set(wd.strip() for wd in r)
def isreal(to_check:str) -> bool:
    return to_check in real_set


def nonsense(s:int, plow, pup) -> str:
    while True:
        word = ''.join(choice(ascii_lowercase) for _ in range(s))
        if plow <= predict(word) <= pup and not isreal(word):
            return word

def nonsentence(p_lower, p_upper) -> str:
    return ' '.join(nonsense(randint(3,7), p_lower, p_upper) for _ in range(10))

In [13]:
for _ in range(5):
    print(nonsentence(0, .2))

yxafno fwzh dfvs brzamq mgnoev jqkgqca ywqruwe jsab veelrl bsb
cmcc eyb ywafiq trpw bfw qqrk rvkzlrk jcurc ckcd cmdazd
gwci xvws uyaui lwydiiy vdq yyzmitt znzyqxj ogvvo iacjvzf qxm
govv xhdolr nygy bxw otu pwqaey ccel bhxl xwlwii dzg
jcjtu bgkbq bicdvuj fxtngs thwe nbtb xejmwrq tovbdgo cggp glztsuj


In [14]:
out:str = ''
for _ in range(7):
    out += '\n'
    out += nonsentence(.95, 1)
    
print(out)


cudpeus mipheg umed oud dashal onfpred ysoun sbass veacils boyed
dricgs lrow ose muvian rehden gooish poom oaous ise ded
oue eidee smoct topber dubsoid dium botin uubose ezeles mafs
ead rogon oms ond tong taice eradock folurah colviy hogsic
iostirg etusint toxsed lepdy baopan ped oraimea lnogley tule foitls
moal ejaisey sedlan momly geins woss hilwerk doiks ivins ows
iad erhwads homar ubrnins ilisk tolies hlampte snocera vowe ochafe


In [18]:
ostr = []

for _ in range(100):
    word = nonsense(4, .95, 1)
    ostr.append(word)
    
print(' '.join(ostr))

130000
cide bsed chay bulp wawe boit tofs groo erak cunk duad bape jash wrom hims teus faly crok ruas jamo hult quts ghan stty rral asic tant dufs flld mian prok ghas foan tway calp jous sapy lomy ctar gaks elut fogl coth domb wovo grus vomb liat wino lelk megs dapt tock inge wism cuge maln fulk ched reep ough dain wifs kned rede cace cogy ppat aset utal frub llar coar pung oods couy adst luat anin trub toom afus qurp mure gass golo aray sifs tala durb smup wack esen bive samb rted brys mang scuk balf


In [8]:
lorem = '''Lorem ipsum dolor sit amet, consectetur adipiscing elit. Phasellus laoreet eget ante a fringilla. Fusce eros felis, volutpat at varius sit amet, consectetur id purus. Mauris scelerisque pharetra est, vel mattis mi euismod a. Sed vestibulum sed sapien sollicitudin rutrum. Vivamus vitae augue nulla. Nullam at consectetur nisl. In laoreet, diam vitae luctus laoreet, dolor metus sollicitudin quam, et lobortis velit enim eu dolor. Vivamus ac odio dui. Aliquam pulvinar nisl urna, non dictum odio dapibus a. Sed luctus dictum tortor imperdiet maximus.'''

def l_clean(l_word:str) -> str:
    return ''.join(l_char for l_char in l_word.lower() if l_char in ascii_lowercase)



for l_word in lorem.split():
    l_word = l_clean(l_word)
    print(f'{predict(l_word):.2f} {l_word}')

1.00 lorem
1.00 ipsum
1.00 dolor
1.00 sit
1.00 amet
1.00 consectetur
1.00 adipiscing
1.00 elit
1.00 phasellus
1.00 laoreet
1.00 eget
1.00 ante
1.00 a
1.00 fringilla
1.00 fusce
1.00 eros
1.00 felis
1.00 volutpat
1.00 at
1.00 varius
1.00 sit
1.00 amet
1.00 consectetur
1.00 id
1.00 purus
1.00 mauris
1.00 scelerisque
1.00 pharetra
1.00 est
1.00 vel
1.00 mattis
1.00 mi
1.00 euismod
1.00 a
1.00 sed
1.00 vestibulum
1.00 sed
1.00 sapien
1.00 sollicitudin
1.00 rutrum
1.00 vivamus
1.00 vitae
1.00 augue
1.00 nulla
1.00 nullam
1.00 at
1.00 consectetur
1.00 nisl
1.00 in
1.00 laoreet
1.00 diam
1.00 vitae
1.00 luctus
1.00 laoreet
1.00 dolor
1.00 metus
1.00 sollicitudin
1.00 quam
1.00 et
1.00 lobortis
1.00 velit
1.00 enim
1.00 eu
1.00 dolor
1.00 vivamus
1.00 ac
1.00 odio
1.00 dui
1.00 aliquam
1.00 pulvinar
1.00 nisl
1.00 urna
1.00 non
1.00 dictum
1.00 odio
1.00 dapibus
1.00 a
1.00 sed
1.00 luctus
1.00 dictum
1.00 tortor
1.00 imperdiet
1.00 maximus


In [21]:
for mterm in (('uair', 'dair', 'fair', 'nair', 'zair', 'swanton', 'upb',
               'jiggly', 'jigglypuff', 'ganondorf', 'douglas',
               'pichu', 'pikachu', 'melee', 'mewtwoking',
               'teambeer', 'smokejoints', 'bigdoinks', 'amish', 'dwarf',
               'sean', 'rosebaugh', 'rosebagel', 'rosenbaum')):
    print(f'{predict(mterm):.3f} {mterm}')

0.156 uair
0.923 dair
0.918 fair
0.705 nair
0.755 zair
0.997 swanton
0.083 upb
0.997 jiggly
0.015 jigglypuff
0.805 ganondorf
0.996 douglas
0.924 pichu
0.081 pikachu
0.982 melee
0.995 mewtwoking
1.000 teambeer
0.924 smokejoints
0.410 bigdoinks
0.808 amish
0.823 dwarf
0.995 sean
0.997 rosebaugh
0.999 rosebagel
0.998 rosenbaum


In [ ]:
for mterm in (('height', 'omnipointment', 'tumpus', 'scrungo', 'muncho',
               'plumbus',)):
    print(f'{predict(mterm):.3f} {mterm}')

2.049262762069702


In [51]:
def make_change(coins, n):
    stack = [1] + [0]*n
    
    for coin in coins:
        for j in range(coin, n+1):
            stack[j] += stack[j-coin]
            
    return stack[-1]

start = time()
make_change([12, 15, 17, 22, 31, 67, 99, 123, 135, 868], 999999)
print(time()-start)

2.049262762069702


In [50]:
@numba.jit
def make_change(coins, n):
    stack = [1] + [0]*n
    
    for coin in coins:
        for j in range(coin, n+1):
            stack[j] += stack[j-coin]
            
    return stack[-1]

start = time()
make_change([12, 15, 17, 22, 31, 67, 99, 123, 135, 868], 999999)
print(time()-start)

2.049262762069702
